# Análise dos Tópicos 

Esse notebook possui os códigos para a análise dos tópicos extraídos dos diários oficiais. 

## Imports Necessários

In [1]:
# Código para poder usar os módulos implementados nesse repositório no notebook 
import sys, os
path_module = os.path.abspath('../modules/')
if path_module not in sys.path:
    sys.path.append(path_module)

In [2]:
# imports do projeto

from utils.process_gazette import ProcessGazette
from preprocess.pre_process_text import PreProcessText
from nlp.extract_topics import ExtractTopics
from nlp.ner import Ner

## Extração e análise dos tópicos do Diário Teste

In [24]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
all_pages_name = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=False)
    all_pages.append(pages)
    all_pages_name.append(txt_file)

In [25]:
revistas_filtradas = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")

for gazette in range(len(all_pages)):
    revistas_filtradas[gazette] = {}

    for page, text in all_pages[gazette].items():
        topics_[page] = {'text': text}
        tokens_ = ppt.process_text(topics_[page]['text'])
        for token in tokens_:
            if len(tokens_) > 5 and page and token in ['aquisicao', 'contratacao', 'emergencial', 'dispensa', 'licitacao','compra','comprar','licitatorio', 'contrato']:
                revistas_filtradas[gazette][page] = topics_[page]
                break

In [26]:
for i in range(0,len(revistas_filtradas)):
    revistas_filtradas[i] = list(revistas_filtradas[i].keys())

### resultado da filtragem.

### Extraindo de multiplas gazetas

In [3]:
pp = ProcessGazette(BASE_DIR="gazettes/")
txt_files = [f for f in os.listdir("gazettes/") if f.endswith('.txt')]
all_pages = []
for txt_file in txt_files:
    pages = pp.break_pages(txt_file, "ANO [X|V|I]+ ", save_file=True)
    all_pages.append(pages)

In [4]:
cont = 1 #len(revistas_filtradas)

In [27]:
all_topics = {}
topics_ = {}
ppt = PreProcessText("pt_core_news_lg")
for gazette in range(0,cont):
    all_topics[gazette] = {}
    for page, text in all_pages[gazette].items():
        
        extractor = ExtractTopics(model = 'default')
        #extractorGpt = ExtractTopics(model = 'gpt')
        extractorZeroshot = ExtractTopics(model = 'zeroshot')
        extractorMaritalk = ExtractTopics(model = 'maritalk')
        ner = Ner()
        topics_[page] = {'text': text, 'topics': []}
        tokens_ = ppt.process_text(topics_[page]['text'])
        if len(tokens_) > 5 and page:
            topics_[page]['tokens'] = tokens_
            topics_[page]['topics'], topics_[page]['topics_info'] = extractor.extract_topics(topics_[page]['tokens'])
            #topics_[page]['topicsGpt'], topics_[page]['topics_infoGpt'] = extractorGpt.extract_topics(topics_[page]['tokens'])
            topics_[page]['topicsZeroshot'], topics_[page]['topics_infoZeroshot'] = extractorZeroshot.extract_topics(topics_[page]['tokens'])
            topics_[page]['topicsMaritalk'], topics_[page]['topics_infoMaritalk'] = extractorMaritalk.extract_topics(topics_[page]['tokens'])
            
            topics_[page]['Ner'] = ner.extract_entities_tokens(topics_[page]['tokens'])
            topics_[page]['Ner_text'] = ner.extract_entities_text(topics_[page]['text'])
            #topics_[page]['Ner_mari_txt'] = ner.extract_entities_maritalk(topics_[page]['text'])
            #topics_[page]['Ner_mari'] = ner.extract_entities_maritalk(topics_[page]['tokens'])
            topics_[page]['similarity'] = extractor.find_topics('Licitação')
            all_topics[gazette][page] = topics_[page]


### run

In [28]:
gazeta = 0
pg = 4

In [15]:
words = all_topics[gazeta][pg]['text'].split()
new_text = ""
for i, word in enumerate(words):
    new_text += word + " "
    if (i + 1) % 20 == 0:
        new_text += "\n"

print(new_text)

| N º 7.6114 Art. 13. As ações previstas neste Capítulo têm caráter temporário e excepcional, devendo perdurar durante o 
período da pandemia. Parágrafo único. Fica autorizada a prorrogação de ofício do termo de colaboração ou fomento, acordos de cooperação, 
termos de parceria, contratos de repasse, contratos de gestão, convênios e demais ajustes similares com vencimento a partir de 19 
de março de 2020, data de decretação do estado de emergência no Município de Salvador, pelo prazo de duração da 
emergência até 3 (três) meses após a decretação do fim da emergência, nas mesmas condições avençadas, devendo ser observados os 
demais aspectos procedimentais da legislação pertinente ao ajuste prorrogado. Art. 14. Fica autorizado o estabelecimento, mediante Decreto, de regras excepcionais 
em relação à prestação de contas quanto aos ajustes que sofram as modificações previstas no art. 10 desta Lei, desde 
que justificadas ante o cenário de emergência e que estejam de acordo com a legislaçã

In [29]:
all_topics[gazeta][pg]['topics_info'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,104,-1_acordo_objetivo_compor_finalidade,"[acordo, objetivo, compor, finalidade, convida...","[acordo, compor, acordo]"
1,0,23,0_urbanismo___,"[urbanismo, , , , , , , , , ]","[municipal, urbanismo, urbanismo]"
2,1,19,1_entidade_instituicao_gabinete_desenvolvimento,"[entidade, instituicao, gabinete, desenvolvime...","[entidade, entidade, entidade]"
3,2,18,2_secretaria_geral_secretario_,"[secretaria, geral, secretario, , , , , , , ]","[secretario-geral, Secretaria, Secretaria]"
4,3,18,3_situacao_indireto_calamidade_conforme,"[situacao, indireto, calamidade, conforme, sim...","[indireto, indireto, situacao]"


In [30]:
all_topics[gazeta][pg]['topics_infoMaritalk'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,133,-1_coordenador_coordenacao_municipio_emergencia,"[coordenador, coordenacao, municipio, emergenc...","[coordenador, coordenador, coordenador]"
1,0,32,0_autorizar_convidar_auxiliar_qualquer,"[autorizar, convidar, auxiliar, qualquer, moni...","[autorizar, autorizar, autorizar]"
2,1,30,1_operacao_promocao_administracao_articulacao,"[operacao, promocao, administracao, articulaca...","[operacao, operacao, operacao]"
3,2,28,2_secretario_secretaria_secretariogeral_gestao,"[secretario, secretaria, secretariogeral, gest...","[secretario, secretario, secretario]"
4,3,25,3_institucional_indicar_instituicao_indireto,"[institucional, indicar, instituicao, indireto...","[institucional, institucional, institucional]"


In [31]:
all_topics[gazeta][pg]['topics_infoZeroshot'][:5]

,Topic,Count,Name,Representation,Representative_Docs
0,0,37,Dispensas de Licitação,"[publico, informacao, resultado, modificacao, ...","[publico, publico, Publico]"
1,1,7,Contratações Emergenciais,"[emergencia, , , , , , , , , ]","[emergencia, emergencia, Emergencia]"
2,2,5,Aquisições Diretas,"[diretor, direto, direito, diretriz, , , , , , ]","[direto, diretor, diretor]"
3,3,59,-1_membro___,"[membro, , , , , , , , , ]","[membro, membro, membro]"
4,4,29,0_indicar_indireto_inspetor_,"[indicar, indireto, inspetor, , , , , , , ]","[indicar, indicar, indicar]"


In [11]:
all_topics[gazeta][pg]['Ner']

[(periodo, 'PER'),
 (gestao convenio, 'LOC'),
 (modificacao, 'LOC'),
 (instituicao comite, 'ORG'),
 (Educacao, 'LOC'),
 (Camara Municipal, 'PER'),
 (Ministerio Publico, 'ORG'),
 (Brasil, 'LOC'),
 (Centro, 'LOC'),
 (Guarda, 'LOC'),
 (Secretaria Municipal, 'LOC'),
 (Secretaria Municipal, 'LOC'),
 (Centro, 'LOC'),
 (Centro, 'LOC')]

In [12]:
all_topics[gazeta][pg]['Ner_text']

[(Capítulo, 'MISC'),
 (Município de Salvador, 'LOC'),
 (Decreto, 'MISC'),
 (Lei, 'MISC'),
 (CAPÍTULO V DO COMITÊ DE INTEGRAÇÃO INSTITUCIONAL  Art. 15, 'MISC'),
 (Comitê de Integração Institucional, 'ORG'),
 (Município de Salvador, 'LOC'),
 (Covid-19, 'LOC'),
 (Comitê de Integração Institucional, 'ORG'),
 (I - Prefeito Municipal, 'MISC'),
 (II - Vice-Prefeito, 'MISC'),
 (Casa Civil, 'PER'),
 (IV - Chefe de Gabinete do Prefeito, 'MISC'),
 (V - Procurador-Geral do Município, 'MISC'),
 (Secretário Municipal da Saúde, 'PER'),
 (Secretário Municipal da Fazenda, 'LOC'),
 (Secretário Municipal de Promoção Social e Combate à Pobreza, 'PER'),
 (Secretário Municipal de Educação, 'PER'),
 (Secretário Municipal de Desenvolvimento e Urbanismo, 'LOC'),
 (XIV- Secretário Municipal de Ordem Pública, 'PER'),
 (Controlador-Geral do Município, 'LOC'),
 (Comitê de Integração Institucional, 'ORG'),
 (Presidente, 'MISC'),
 (Secretários Municipais, 'PER'),
 (Administração Indireta Municipal, 'ORG'),
 (Prefeit

In [15]:
print(all_topics[gazeta][pg]['Ner_mari_txt'])

  ```python
{
  "Person": ["Prefeito Municipal", "Vice-Prefeito", "Chefe da Casa Civil", "Chefe de Gabinete do Prefeito", "Procurador-Geral do Município", "Secretário Municipal da Saúde", "Secretário Municipal da Fazenda", "Secretário Municipal de Promoção Social e Combate à Pobreza", "Secretário Municipal de Educação", "Secretário Municipal de Infraestrutura", "Secretário Municipal de Gestão", "Secretário Municipal de Mobilidade", "Secretário Municipal de Desenvolvimento e Urbanismo", "Secretário Municipal de Ordem Pública", "Secretário Municipal de Comunicação", "Secretário-Geral de Articulação Comunitária e Prefeituras-Bairro", "Controlador-Geral do Município"],
  "Organization": ["Prefeitura Municipal de Salvador", "Centro de Coordenação de Operações"],
  "Location": ["Salvador"],
  "Date": ["19 de março de 2020", "perdurar durante o período da pandemia", "3 (três) meses"],
  "Time": ["durante o estado de emergência", "estado de calamidade pública decretados em função do Covid-19"]

In [17]:
print(all_topics[gazeta][pg]['Ner_mari'])

  ```python
{
    "Person": ["prever", "capitulo", "carater", "temporario", "excepcional", "perdurar", "durante", "periodo", "pandemia", "paragrafo", "autorizar", "prorrogacao", "oficio", "colaboracao", "fomento", "acordo", "cooperacao", "termos", "parceria", "contrato", "repasse", "contrato", "gestao", "convenio", "ajuste", "similar", "vencimento", "decretacao", "emergencia", "municipio", "salvador", "duracao", "Emergencia", "decretacao", "emergencia", "condicao", "avencadas", "observar", "aspecto", "procedimental", "legislacao", "pertinente", "ajuste", "prorrogar", "autorizar", "estabelecimento", "mediante", "decreto", "excepcional", "prestacao", "ajuste", "sofram", "modificacao", "prever", "justificar", "cenario", "emergencia", "acordo", "legislacao", "vigente", "capitulo", "comite", "integracao", "institucional", "reconhecido", "instituicao", "comite", "integracao", "institucional", "finalidade", "centralizar", "tomada", "decisao", "integrar", "alinhar", "governamental", "enfrentam